### **Pede-se nesta prova prática que o candidato treine um classificador binário incorporando o conceito de stacking em sua solução, e que esteja apto a justificar as escolhas que foram feitas ao longo do desenvolvimento.**

In [1]:
import sweetviz as sv
import numpy as np
import pandas as pd
import sklearn
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import lazypredict
from lazypredict.Supervised import LazyClassifier

/home/melanie/anaconda3/envs/case_II/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from mlens.ensemble import SuperLearner

[MLENS] backend: threading


**Analisando o conjunto de treinamento**

In [3]:
# X_train.csv
df_X_train = pd.read_csv("X_train.csv", sep=",", header=None)
print(f'Tamanho X_train: {len(df_X_train)}')
df_X_train.head()

Tamanho X_train: 800


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.36,-1.51,0.29,1.95,1.36,0.87,0.87,-1.51,1.95,-1.76,-0.38,0.51,-0.16,-1.60,1.36,-3.98,-1.60,-4.40,1.89,0.95
1,-0.90,-0.48,-1.67,-2.28,-0.90,-1.08,-1.08,-0.48,-2.28,-0.47,1.14,1.06,0.19,-2.88,-0.90,-0.43,-2.88,1.69,1.80,2.77
2,-2.32,-3.35,0.61,0.43,-2.32,-0.39,-0.39,-3.35,0.43,-5.14,0.16,4.49,-2.40,-7.27,-2.32,-3.39,-7.27,-4.42,2.17,4.53
3,0.44,-0.36,0.23,-0.08,0.44,2.36,2.36,-0.36,-0.08,-0.45,-1.46,0.54,-2.65,0.84,0.44,-0.76,0.84,-4.42,1.36,0.33
4,-1.55,-0.54,2.41,2.21,-1.55,-1.11,-1.11,-0.54,2.21,-1.90,0.06,1.88,0.17,-3.14,-1.55,-0.88,-3.14,-1.19,-1.03,-0.44


In [4]:
# Usando sweetviz para analisar a distribuição das variáveis individualmente
# report_X_train= sv.analyze(df_X_train)
# report_X_train.show_html('df_X_train.html')

In [5]:
# Preenchendo os valores NaN
df_X_train = df_X_train.fillna(df_X_train.median())

# Eliminando os outliers
z_scores = stats.zscore(df_X_train)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
df_X_train_unlab = df_X_train[filtered_entries]

print(f'Tamanho X_train: {len(df_X_train_unlab)}')
df_X_train_unlab.head()

Tamanho X_train: 770


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.36,-1.51,0.29,1.95,1.36,0.87,0.87,-1.51,1.95,-1.76,-0.38,0.51,-0.16,-1.60,1.36,-3.98,-1.60,-4.40,1.89,0.95
1,-0.90,-0.48,-1.67,-2.28,-0.90,-1.08,-1.08,-0.48,-2.28,-0.47,1.14,1.06,0.19,-2.88,-0.90,-0.43,-2.88,1.69,1.80,2.77
3,0.44,-0.36,0.23,-0.08,0.44,2.36,2.36,-0.36,-0.08,-0.45,-1.46,0.54,-2.65,0.84,0.44,-0.76,0.84,-4.42,1.36,0.33
4,-1.55,-0.54,2.41,2.21,-1.55,-1.11,-1.11,-0.54,2.21,-1.90,0.06,1.88,0.17,-3.14,-1.55,-0.88,-3.14,-1.19,-1.03,-0.44
5,-0.85,-0.35,-0.62,-0.37,-0.85,-0.34,-0.34,-0.35,-0.37,-1.00,-0.13,0.46,0.04,-1.39,-0.85,0.12,-1.39,0.29,0.48,1.81


In [6]:
# report_X_train_unlab = sv.analyze(df_X_train_unlab)
# report_X_train_unlab.show_html('report_X_train_unlab.html')

**Analisando o conjunto de teste**

In [7]:
# X_test.csv
df_X_test = pd.read_csv("X_test.csv", sep=",", header=None)
print(f'Tamanho X_test: {len(df_X_test)}')
df_X_test.head()

Tamanho X_test: 200


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.62,-1.63,-3.05,-1.37,-0.62,0.82,0.82,-1.63,-1.37,-2.52,-1.07,0.00,-0.90,-1.01,-0.62,0.02,-1.01,-0.74,1.86,5.28
1,0.48,-2.69,1.93,2.19,0.48,-1.37,-1.37,-2.69,2.19,-2.65,1.97,2.99,0.22,-5.59,0.48,-5.64,-5.59,-3.26,1.29,0.06
2,-0.76,0.62,3.10,2.37,-0.76,0.61,0.61,0.62,2.37,-0.04,-1.19,0.32,-1.10,1.74,-0.76,1.70,1.74,-1.50,-2.98,-3.35
3,1.55,-0.82,0.45,1.95,1.55,2.58,2.58,-0.82,1.95,-0.99,-1.92,-0.59,-1.63,2.05,1.55,-1.64,2.05,-5.17,0.60,-0.24
4,-1.59,1.55,3.49,3.63,-1.59,1.00,1.00,1.55,3.63,-0.54,-2.79,-0.31,-0.62,2.11,-1.59,2.71,2.11,-2.21,-3.31,-2.81


In [8]:
# report_X_test = sv.analyze(df_X_test)
# report_X_test.show_html('report_X_test.html')

In [9]:
# Y_test.csv
df_Y_test = pd.read_csv("y_test.csv", sep=",", header=None)
print(f'Tamanho X_test: {len(df_Y_test)}')
df_Y_test.head()

Tamanho X_test: 200


,0
0,1.00
1,0.00
2,1.00
3,0.00
4,1.00


In [10]:
# report_Y_test = sv.analyze(df_Y_test)
# report_Y_test.show_html('report_Y_test.html')

## **Semisupervised Learning usando Propagação dos Labels**

Reservando 70% do teste para teste e usando 30% do teste para adicionar data rotulada no treino.

In [11]:
# Dividindo o conjunto de teste em duas partes
df_X_train_lab, df_X_test_small, df_y_train_lab, df_y_test_small = train_test_split(df_X_test, df_Y_test, test_size=0.70, random_state=1, stratify=df_Y_test)

# Resetando os índices
df_X_train_lab = df_X_train_lab.reset_index(drop=True)
df_y_train_lab = df_y_train_lab.reset_index(drop=True)
df_X_test_small = df_X_test_small.reset_index(drop=True)
df_y_test_small = df_y_test_small.reset_index(drop=True)

In [12]:
# Resumindo
print('Conjunto de treino UNLABELED e sem outliers:', df_X_train_unlab.shape)
print('Conjunto de treino LABELED:', df_X_train_lab.shape, df_y_train_lab.shape)
print('Conjunto reservado para TESTE:', df_X_test_small.shape, df_y_test_small.shape)

Conjunto de treino UNLABELED e sem outliers: (770, 20)
Conjunto de treino LABELED: (60, 20) (60, 1)
Conjunto reservado para TESTE: (140, 20) (140, 1)


In [13]:
df_y_train_lab[0].value_counts()

0.00    30
1.00    30
Name: 0, dtype: int64

In [14]:
df_y_test_small[0].value_counts()

1.00    71
0.00    69
Name: 0, dtype: int64

In [15]:
# Criando "y_train_unlab" preenchido com "-1"
y_train_unlab = [-1 for _ in range(len(df_X_train_unlab))]
df_y_train_unlab = pd.DataFrame(y_train_unlab)

# Misturando o X de treino com labels e sem labels
df_X_train_misturado = pd.concat([df_X_train_unlab, df_X_train_lab])
# Misturando o Y de treino com labels e sem labels
df_y_train_misturado = pd.concat([df_y_train_unlab, df_y_train_lab])

print('Conjunto de treino MISTURADO:', df_X_train_misturado.shape, df_y_train_misturado.shape)
df_y_train_misturado = df_y_train_misturado.astype(int)

Conjunto de treino MISTURADO: (830, 20) (830, 1)


In [ ]:
# Convertendo as entradas do modelo para NumPy Array, https://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.LabelPropagation.html
X_train_misturado = df_X_train_misturado.to_numpy()
print(X_train_misturado.shape)
y_train_misturado = df_y_train_misturado.to_numpy()
y_train_misturado = np.ravel(y_train_misturado)
print(y_train_misturado.shape)
X_test_small = df_X_test_small.to_numpy()
y_test_small = df_y_test_small.to_numpy()
y_test_small = np.ravel(y_test_small)

### **Label Propagation**

In [ ]:
def rbf_kernel_safe(X, Y=None, gamma=None): 

      X, Y = sklearn.metrics.pairwise.check_pairwise_arrays(X, Y) 
      if gamma is None: 
          gamma = 1.0 / X.shape[1] 

      K = sklearn.metrics.pairwise.euclidean_distances(X, Y, squared=True) 
      K *= -gamma 
      K -= K.max()
      np.exp(K, K)    # exponentiate K in-place 
      return K 

In [ ]:
model1 = LabelPropagation(kernel = rbf_kernel_safe, gamma=20, max_iter=5000)
model1.fit(X_train_misturado, y_train_misturado)

tran_labels = model1.transduction_

## **Pegando os rótulos estimados e ajustando modelos de aprendizado supervisionado**

### Testando um modelo individual: **LogisticRegression**

In [ ]:
model2 = LogisticRegression()
model2.fit(X_train_misturado, tran_labels)

yhat2 = model2.predict(X_test_small)
score2 = accuracy_score(y_test_small, yhat2)
print('AUC: %.3f' % (score2*100))

## Testando um **Super Learner** usando a livraria mlens

In [ ]:
# Lista de modelos base
def get_models():
    models = list()
    models.append(LogisticRegression(solver='liblinear'))
    models.append(DecisionTreeClassifier())
    models.append(SVC(gamma='scale', probability=True))
    models.append(GaussianNB())
    models.append(KNeighborsClassifier())
    models.append(AdaBoostClassifier())
    models.append(BaggingClassifier(n_estimators=10))
    models.append(RandomForestClassifier(n_estimators=10))
    models.append(ExtraTreesClassifier(n_estimators=10))
    return models
 
# Criando o super learner
def get_super_learner(X):
    ensemble = SuperLearner(scorer=accuracy_score, folds=10, shuffle=True, sample_size=len(X))
    # add base models
    models = get_models()
    ensemble.add(models)
    # add the meta model
    ensemble.add_meta(LogisticRegression(solver='lbfgs'))
    return ensemble
 

ensemble = get_super_learner(X_train_misturado)
ensemble.fit(X_train_misturado, tran_labels)
print(ensemble.data)
yhat4 = ensemble.predict(X_test_small)
print('AUC: %.3f' % (accuracy_score(y_test_small, yhat4) * 100))